# Carbon stock

In [ ]:
import pandas as pd

In [ ]:
path_data = '../data/ISRIC_report_2008_02_csv/'
path_horizon = f'{path_data}/WISE3_HORIZON.csv'

horizon = pd.read_csv(path_horizon)
horizon.columns = [c.lower() for c in horizon.columns]

In [ ]:
horizon.head()

,wise3_id,honu,desig,topdep,botdep,mcolor,dcolor,orgc,totn,caco3,...,cecsoil,bsat,sand,silt,clay,gravel,bulkdens,vmc1,vmc2,vmc3
0,AF0001,1,NaN,0,15,10YR4/3,10YR4/2,7.6,0.6,93.0,...,NaN,NaN,40.0,40.0,20.0,20.0,NaN,NaN,NaN,NaN
1,AF0001,2,NaN,15,60,10YR5/3,10YR5/2,2.3,0.3,177.0,...,NaN,NaN,10.0,55.0,35.0,NaN,NaN,NaN,NaN,NaN
2,AF0001,3,NaN,60,150,10YR5/3,NaN,0.9,0.3,182.0,...,NaN,NaN,10.0,55.0,35.0,NaN,NaN,NaN,NaN,NaN
3,AF0002,1,NaN,0,20,10YR4/2,NaN,12.8,0.9,191.0,...,NaN,NaN,40.0,40.0,20.0,NaN,NaN,NaN,NaN,NaN
4,AF0002,2,NaN,20,60,10YR4/2,NaN,6.0,0.6,192.0,...,NaN,NaN,15.0,65.0,20.0,NaN,NaN,NaN,NaN,NaN


`orgc` = $c$. $[c] =  \frac{g}{kg}.$  
`bulkdens` = $b$. $[b] = \frac{g}{cm^3}.$  
`botdep - topdep` = $l$. $[l] = cm.$  


Depth of interest:
$$
L = 100 \: cm
$$

Method of calculation:
1. Clip off all segments below desired depth.
2. Calculate the carbon volume density of each segment.
3. For each profile, average its segments' carbon volume densities, weighted by segment length.

i.e. for a profile with two segments:
$$
\frac{l_1}{L} c_1 b_1 + \frac{l_2}{L} c_2 b_2 \quad
$$
with units:
$$
\frac{g}{10^{-3} m^{3}} = \frac{kg}{m^{3}} \quad.
$$

In [ ]:
# Depth of interest.
depth = 100 # cm

In [ ]:
# Select organic carbon-related columns.
cols = ['wise3_id', 'topdep', 'botdep', 'orgc', 'bulkdens']
df = horizon[cols].copy()

Get $\frac{l}{L}$.

In [ ]:
# Clip bottoms at desired depth.
# Calculate length of each segment.
# Normalize by depth of interest.
df.loc[:, 'botdep_trunc'] = df['botdep'].clip(upper=depth)
df.loc[:, 'segment_length'] = df['botdep_trunc'] - df['topdep']
df.loc[:, 'segment_length'] /= depth

In [ ]:
# Select needed columns to continue.
df = df[['wise3_id', 'segment_length', 'orgc', 'bulkdens']]

In [ ]:
# Ignore entries with no valid bulkdens, orgc, or segment_length.
df = df[
    df.bulkdens.notnull() & df.orgc.notnull() & df.segment_length.gt(0)
]

Get $\frac{l}{L} c b$.

In [ ]:
df.loc[:, 'weighted_orgc_voldens'] = (
    df['segment_length'] * df['orgc'] * df['bulkdens']
)

In [ ]:
# Average segments weighted by their length for each profile
df = df.groupby('wise3_id').weighted_orgc_voldens.sum()
df = pd.DataFrame(df)
df.rename({'weighted_orgc_voldens': 'carbon_stock'}, axis=1, inplace=True)

Carbon stock.  Units: $\frac{kg}{m^3}$

In [ ]:
df

,carbon_stock
wise3_id,
AL0007,12.11205
AL0008,6.79968
AL0009,10.18997
AL0010,15.78198
AL0011,17.50674
...,...
ZW0060,12.94279
ZW0062,10.39402
ZW0063,11.58268


In [ ]:
df.loc['TH0285']

carbon_stock    12.78894
Name: TH0285, dtype: float64